In [3]:
import pandas as pd
import numpy as np 

In [1]:
comment1 ='''
bow : 단어의 순서 고려x, 단어의 빈도수를 기반으로 만든 행렬
1) 단어 추출( 단어 집합 ) -> 단어에 대해 정수 인덱스 설정  
2) 단어의 등장 횟수를 저장한 벡터를 생성
'''

In [6]:
text = '정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.'

In [15]:
# 퀴즈 text에서 .마침표를 제거
import re
token = re.sub('\.','',text)

In [14]:
from konlpy.tag import Okt

In [17]:
okt = Okt()
token = okt.morphs(token)

In [18]:
token

['정부', '가', '발표', '하는', '물가상승률', '과', '소비자', '가', '느끼는', '물가상승률', '은', '다르다']

In [19]:
word2index = {} # 정수 인덱스를 저장 { 정수 : 인덱스 번호 , ..}
bow = [] # [단어들 마다의 빈도수가 저장]
for v in token:
    if v not in word2index.keys():
        word2index[v] = len(word2index) # 0 ~ 9까지
        bow.insert(len(word2index)-1,1)
    else:
        idx =word2index[v]
        bow[idx] = bow[idx]+1
print(word2index)

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}


In [20]:
print(bow)

[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [21]:
# 전체 문서 (n개 문서) ={d1,d2,d3 ...}
# 단어집합 (m개 문서) = {t1,t2,t3 ...}
# Xij = 문서 di에 있는 단어 tj의 출현 빈도수

In [25]:
# 사이킷런에는 단어를 벡터로 만들어주는 클래스가 있다.
from sklearn.feature_extraction import DictVectorizer # ~ Vectorizer는 bow를 만들어내는 클래스들이다
# DictVectorizer 문서에서 단어의 빈도가 저장된 딕션너리 -> bow 생성
v = DictVectorizer(sparse = False) # sparse  = True  : 텍스트 데이터를 압축해서 bow 생성 ( 메모리 효율 )
d=[{'a':1,'b':2},{'b':3,'c':1}] # [ 문서1, 문서2 ] :{ 단어1 : 빈도수, 단어2 : 빈도수 } {문서에 등장하는 단어}
d

[{'a': 1, 'b': 2}, {'b': 3, 'c': 1}]

In [27]:
v.fit_transform(d) # 이것이 bow이다.

array([[1., 2., 0.],
       [0., 3., 1.]])

In [28]:
v.feature_names_ # 문서를 구성하는 열의 이름이 출력되어진다. 

['a', 'b', 'c']

In [29]:
# bow에 새로운 문서 데이터를 전달하여 변환
v.transform({'c':5,'d':2}) # 기존에 bow는 d가 없다. a,b,c,만 존재 
# d는 반영되지 않은 결과를 보여준다 a:0 b:0 c:5 열만 보여준다. 

array([[0., 0., 5.]])

In [30]:
# CountVectorizer 
# 1) 문서 -> 단어 토큰으로 변환해준다.
# 2) 토큰의 빈도수 계산
# 3) bow 벡터 생성
from sklearn.feature_extraction.text import CountVectorizer
corpus = ['This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?']

In [31]:
v = CountVectorizer()
v.fit(corpus)
v.vocabulary_ # 단어 인덱스 부여 번호 확인

{'this': 9,
 'is': 3,
 'the': 7,
 'first': 2,
 'document': 1,
 'second': 6,
 'and': 0,
 'third': 8,
 'one': 5,
 'last': 4}

In [33]:
v.transform(corpus).toarray() # 문장의 bow를 생성한 결과 열 순서가 단어 인덱스 0 : this 1: documment ....
# 'This is the first document.'

array([[0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]], dtype=int64)

In [35]:
# 새로운 문서가 입력 시 -> bow 변환
v.transform(['this is the second document']).toarray() # 크기가 10인 array 

array([[0, 1, 0, 1, 0, 0, 1, 1, 0, 1]], dtype=int64)

In [36]:
v.transform(['someting new']).toarray() # 내용이 bow에 포함되지 않기 때문에 0 행렬이 출력

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [37]:
v = CountVectorizer(stop_words=['and','is','the','this'])
v.fit(corpus)
v.vocabulary_ # stop_words 단어가 빠졌다. 관사, 접속사, 조사, => 단어집합을 만들 때 무시가능 

{'first': 1, 'document': 0, 'second': 4, 'third': 5, 'one': 3, 'last': 2}

In [38]:
v = CountVectorizer(stop_words='english') # 기존에 등록된 stop_words
v.fit(corpus)
v.vocabulary_

{'document': 0, 'second': 1}

In [41]:
v = CountVectorizer(analyzer='char') # analyzer 구문 분석기 char : 문자 단위로 
v.fit(corpus)
v.vocabulary_

{'t': 16,
 'h': 8,
 'i': 9,
 's': 15,
 ' ': 0,
 'e': 6,
 'f': 7,
 'r': 14,
 'd': 5,
 'o': 13,
 'c': 4,
 'u': 17,
 'm': 11,
 'n': 12,
 '.': 1,
 'a': 3,
 '?': 2,
 'l': 10}

In [44]:
v = CountVectorizer(token_pattern='t\w+') # bow 문서를 만들 때 소문자 t로 시작하는 단어를 뽀고싶다.
v.fit(corpus)
v.vocabulary_

{'this': 2, 'the': 0, 'third': 1}

In [45]:
v = CountVectorizer(ngram_range=(2,2)) # 최소 2 최대 2인 ngram을 생성한다.
# ngram_range = 의미? 최소 n개 최대 m개의 단어들을 묶어서 1개의 단어로 취급하여 bow생성
# 두 단어가 1개의 단어로 사용
v.fit(corpus)
v.vocabulary_

{'this is': 12,
 'is the': 2,
 'the first': 7,
 'first document': 1,
 'the second': 9,
 'second second': 6,
 'second document': 5,
 'and the': 0,
 'the third': 10,
 'third one': 11,
 'is this': 3,
 'this the': 13,
 'the last': 8,
 'last document': 4}

In [47]:
v = CountVectorizer(token_pattern='t\w+',ngram_range=(1,2)) # 단어 시작이 t이고 ngram이 1에서 2까지일 때 토큰 
v.fit(corpus)
v.vocabulary_

{'this': 3, 'the': 0, 'this the': 4, 'third': 2, 'the third': 1}

In [48]:
v = CountVectorizer(min_df = 2,max_df = 4) # 최소 몇군데 이상에서 등장한 단어 횟수
v.fit(corpus)
v.vocabulary_ , v.stop_words_

({'this': 3, 'is': 2, 'first': 1, 'document': 0},
 {'and', 'last', 'one', 'second', 'the', 'third'})

In [49]:
comment2 ='''
# https://en.wikipedia.org/wiki/Tf%E2%80%93idf#:~:text=In%20information%20retrieval%2C%20tf%E2%80%93idf,in%20a%20collection%20or%20corpus.
tf-idf( 단어 빈도 / 문서 빈도 ): dtm에 있는 각 단어들 마다 중요도를 계산 
# dtm 각각의 문서마다 단어가 몇번 등장하였나 -> 각 단어의 중요도가 없다.
# dtm을 생성 후, tf-idf를 생성 
# tf(d,t) : 문서 d에서 단어 t의 등장 회수 // 단어의 출현 빈도수 
# df(t) : 단어 t가 등장한 문서의 개수(여러번 단어가 등장한 경우에도 1로 카운트)
# idf(d,t) : dt(t)의 반비례수 
# idf(d,t) = log( n / ( 1+df(t) )) : n: 전체 문서의 개수 

log를 취하는 이유
ex) idf(d,t) = 100만 / 1 = 100만
    idf(d,t) = 100만 / 100 = 1만
    idf(d,t) = 100만 / 100만 = 1
    ==> scale이 굉장히 크다
    
    idf(d,t) = log(100만 / 1) = 6
    idf(d,t) = log(100만 / 100) = 4
    idf(d,t) = log(100만 / 100만) = 0
    ==> scale이 적당해짐
    
    로그를 취하지 않으면 희귀한 단어는 엄청난 가중치를 가져옴 -> 로그가 있어 격차가 사라짐
    1+df(t) : 1이 있는 이유 ==> 단어가 한 번도 나오지 않는 경우 분모가 0이 될 수 있기 때문에 방지를 위해 +1
    
    tf-idf 모든 문서에서 자주 등장하는 단어라면 중요도가 낮다고 판단.
    특정 문서에서만 등장하는 단어라면 중요도가 높다고 판다.
    
    tf - idf 값이 낮으면(높으면) -> 단어의 중요도가 낮다(높다)
...

'''

In [ ]:
idf = n/log(1+df(t)) n = 4
tf = 문서 d에서 단어 t의 등장 회수
바나나 = 3 / (4 / log(1+2))

In [51]:
np.log(4/3)  # idf

0.28768207245178085

In [52]:
#tf 1 : 0  2 : 1  3 : 2  4 : 0
0,1*(0.287),2*0.287,0

(0, 0.287, 0.574, 0)

In [53]:
import pandas as pd
from math import log

In [54]:
log(4/3)

0.28768207245178085

In [55]:
docs =[
    '먹고 싶은 사과','먹고 싶은 바나나','길고 노란 바나나 바나나','저는 과일이 좋아요'
]# 문서 4개 
# tf , idf
cv = CountVectorizer()
cv.fit(docs)

CountVectorizer()

In [89]:
df = pd.DataFrame(cv.transform(docs).toarray(),columns =sorted(cv.vocabulary_))
df
# idf(d,t) = log( n / ( 1+df(t) )) : n: 전체 문서의 개수 


,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [87]:
sorted(cv.vocabulary_)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [88]:
cv.vocabulary_

{'먹고': 3,
 '싶은': 6,
 '사과': 5,
 '바나나': 4,
 '길고': 1,
 '노란': 2,
 '저는': 7,
 '과일이': 0,
 '좋아요': 8}

In [90]:
docs =[
    '먹고 싶은 사과','먹고 싶은 바나나','길고 노란 바나나 바나나','저는 과일이 좋아요'
]
cv = CountVectorizer()
cv.fit(docs)
df = pd.DataFrame(cv.transform(docs).toarray(),columns =sorted(cv.vocabulary_))
df*np.log(len(df) / (1+df.apply(lambda x: x>0).sum()))

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147
